In [1]:
pip install langdetect

     |████████████████████████████████| 981 kB 937 kB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=34868e31ebcd468649c81cf3498b60374514f271c3f3a2decc72e1733ef5bf67
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bpemb

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
import re
import nltk
from tqdm import tqdm
import string
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from langdetect import detect
import emoji
from bpemb import BPEmb

In [4]:
# Read DataFrame From Local Path
# Change the path to local path where you store kaggle dataset twcs.csv
df = pd.read_csv("../input/customer-support-on-twitter/twcs/twcs.csv")
df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
...,...,...,...,...,...,...,...
6995,10465,VerizonSupport,False,Wed Nov 01 00:38:00 +0000 2017,@117904 Please check the connection on your sp...,10466,10464.0
6996,10466,117904,True,Wed Nov 01 00:39:19 +0000 2017,@VerizonSupport It’s secured. What’s next?,10467,10465.0
6997,10467,VerizonSupport,False,Wed Nov 01 00:41:33 +0000 2017,@117904 We want to try a reset to your home ne...,10468,10466.0
6998,10468,117904,True,Wed Nov 01 00:42:23 +0000 2017,@VerizonSupport Yes,10469,10467.0


In [5]:
# Create function to filter out text that are not written in English
def keepEnglish(d):
    mask = []
    
    for i,doc in enumerate(d["text"]) :
        try:
            if doc and  doc and 'en' == detect(doc) :
                mask.append(True)
            else:
                mask.append(False)
        except Exception:
            mask.append(False)

    return mask

In [6]:
# Convert to lower case and only keep English words
mask = keepEnglish(df)
df = df[mask]
df['text'] = df["text"].str.lower()
df["text"]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0       @115712 i understand. i would like to assist y...
1           @sprintcare and how do you propose we do that
2       @sprintcare i have sent several private messag...
3       @115712 please send us a private message so th...
5       @115712 can you please send us a private messa...
                              ...                        
6993    @117904 do you happen to know where your main ...
6995    @117904 please check the connection on your sp...
6996           @verizonsupport it’s secured. what’s next?
6997    @117904 we want to try a reset to your home ne...
6999    @117904 please press and hold down the "alarm ...
Name: text, Length: 6618, dtype: object

In [7]:
# Only Keep First asked questions and their reponses
df_q= df[pd.isnull(df.in_response_to_tweet_id) & df.inbound]
df_qa = pd.merge(df_q, df, left_on='tweet_id', right_on='in_response_to_tweet_id')
qa_tweets = df_qa[["text_x","text_y"]]
qa_tweets.columns = ["questions","answers"]
qa_tweets

,questions,answers
0,@sprintcare is the worst customer service,@115712 can you please send us a private messa...
1,@115714 y’all lie about your “great” connectio...,@115713 h there! we'd definitely like to work ...
2,"@115714 whenever i contact customer support, t...",@115715 please send me a private message so th...
3,actually that's a broken link you sent me and ...,@115716 the information pertaining to the acco...
4,"yo @ask_spectrum, your customer service reps a...","@115717 hello, my apologies for any frustratio..."
...,...,...
1767,@115993 i am encountering problems when withdr...,@117899 i am sorry to hear about this trouble ...
1768,@askpaypal sent you guys a dm never got a reply..,"@117900 hey! sorry about the wait, the queue's..."
1769,@askpaypal please see dm,@117901 hi! i've replied to your dm! :) ^jmr
1770,i'm about to close my @115993 account. my debi...,@115993 never offer any solution and thus has ...


In [8]:
bpemb_en = BPEmb(lang="en", dim=100)

def clean_tweet(tweet):
    # Clean Tweet, remove emoji, symbols and tokenize 
    lemmatizer = WordNetLemmatizer()
    english_stopwords = set(stopwords.words("english"))
    digit_chr = re.compile("^[a-zA-Z0-9@_!#$%^&*()<>?/\|}{~:,.]*$")
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_cleaned = []
    for word in tweet_tokens:
        # lemmatize each word and then check pattern before making new sentence
        lemma_word = lemmatizer.lemmatize(word)
        check = digit_chr.match(word)
        if word not in  emoji.UNICODE_EMOJI['en'] and check:
            tweets_cleaned.append(lemma_word)
    #Create new sentence based on old sentence, apply bpe for new sentence
    tweet_clean = " ".join(tweets_cleaned)
    tweet_bpe = bpemb_en.encode(tweet_clean)
    return tweet_bpe

# User counter to create vocab and calculate frequency
def tweet_counter(df):
    count = Counter()
    for tweet in df.values:
        for word in tweet:
            count[word] += 1
        
    return count

# create a index list given a sentence and vocab
def index_tweet(vocab,tweet_list):
    indice = [vocab[x] for x in tweet_list]
    return indice

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:00<00:00, 722116.72B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3784656/3784656 [00:01<00:00, 3381780.06B/s]


In [9]:
# Drop duplicated questions 
qa_tweets.drop_duplicates('questions', inplace=True)
qa_tweets

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,questions,answers
0,@sprintcare is the worst customer service,@115712 can you please send us a private messa...
1,@115714 y’all lie about your “great” connectio...,@115713 h there! we'd definitely like to work ...
2,"@115714 whenever i contact customer support, t...",@115715 please send me a private message so th...
3,actually that's a broken link you sent me and ...,@115716 the information pertaining to the acco...
4,"yo @ask_spectrum, your customer service reps a...","@115717 hello, my apologies for any frustratio..."
...,...,...
1767,@115993 i am encountering problems when withdr...,@117899 i am sorry to hear about this trouble ...
1768,@askpaypal sent you guys a dm never got a reply..,"@117900 hey! sorry about the wait, the queue's..."
1769,@askpaypal please see dm,@117901 hi! i've replied to your dm! :) ^jmr
1770,i'm about to close my @115993 account. my debi...,@115993 never offer any solution and thus has ...


In [10]:
# Create token df after clean tweet 
qa_token = pd.DataFrame(columns = ["question_token", "answer_token"])
qa_token["question_token"] = qa_tweets["questions"].apply(lambda x: clean_tweet(x))
qa_token["answer_token"] = qa_tweets["answers"].apply(lambda x: clean_tweet(x))
qa_token

,question_token,answer_token
0,"[▁is, ▁the, ▁worst, ▁custom, er, ▁service]","[▁can, ▁you, ▁ple, ase, ▁send, ▁u, ▁a, ▁privat..."
1,"[▁y, ▁all, ▁lie, ▁about, ▁your, ▁great, ▁conne...","[▁h, ▁there, ▁, !, ▁defin, it, ely, ▁like, ▁to..."
2,"[▁whe, never, ▁i, ▁contact, ▁custom, er, ▁supp...","[▁ple, ase, ▁send, ▁me, ▁a, ▁private, ▁message..."
3,"[▁actually, ▁a, ▁broken, ▁link, ▁you, ▁sent, ▁...","[▁the, ▁information, ▁per, t, aining, ▁to, ▁th..."
4,"[▁y, o, ▁,, ▁your, ▁custom, er, ▁service, ▁rep...","[▁hel, lo, ▁,, ▁my, ▁ap, ology, ▁for, ▁any, ▁f..."
...,...,...
1767,"[▁i, ▁am, ▁encoun, tering, ▁problem, ▁when, ▁w...","[▁i, ▁am, ▁sor, ry, ▁to, ▁hear, ▁about, ▁this,..."
1768,"[▁sent, ▁you, ▁guy, ▁a, ▁d, m, ▁never, ▁got, ▁...","[▁he, y, ▁, !, ▁sor, ry, ▁about, ▁the, ▁wait, ..."
1769,"[▁ple, ase, ▁see, ▁d, m]","[▁h, i, ▁, !, ▁repl, ied, ▁to, ▁your, ▁d, m, ▁..."
1770,"[▁about, ▁to, ▁close, ▁my, ▁account, ▁., ▁my, ...","[▁never, ▁offer, ▁any, ▁solution, ▁and, ▁thus,..."


In [11]:
# Further checking about  empty questions or answers
qa_token.reset_index(drop=True, inplace = True)
print("Empty questions:")
for i, d in enumerate(tqdm(qa_token['question_token'].values.tolist())):
    if d == []:
        qa_token.drop(i, inplace=True)
        
for i, d in enumerate(tqdm(qa_token['question_token'].values.tolist())):
    if d == []:
        print(i)

qa_token.reset_index(drop=True, inplace = True)
print("")
print("Empty Answers:")        
for i, d in enumerate(tqdm(qa_token['answer_token'].values.tolist())):
    if d == []:
        qa_token.drop(i, inplace=True)
        
for i, d in enumerate(tqdm(qa_token['answer_token'].values.tolist())):
    if d == []:
        print(i)

Empty questions:


100%|██████████| 1704/1704 [00:00<00:00, 785740.33it/s]



Empty Answers:


100%|██████████| 1699/1699 [00:00<00:00, 1188677.65it/s]


In [12]:
# Counter and vocab for question and answers
cnt_q = tweet_counter(qa_token["question_token"])
cnt_a = tweet_counter(qa_token["answer_token"])
vocab_ql = sorted(list(cnt_q))
vocab_al = sorted(list(cnt_a))

In [13]:
# Create final vocab and index lists
vocab = set(vocab_ql).union(set(vocab_al))
vocab = sorted(list(vocab))
vocab_dict = {} 
for i, word in enumerate(vocab): 
    vocab_dict[word] = i   
print(len(vocab))

4028


In [14]:
# Create index list for each sentence for questions and answers
qa_index = pd.DataFrame(columns = ["question_index", "answer_index"])
qa_index["question_index"] = qa_token["question_token"].apply(lambda x: index_tweet(vocab_dict,x))
qa_index["answer_index"] = qa_token["answer_token"].apply(lambda x: index_tweet(vocab_dict,x))
qa_index

,question_index,answer_index
0,"[2511, 3726, 4007, 1808, 366, 3455]","[1559, 4022, 3088, 148, 3443, 3837, 1211, 3141..."
1,"[4016, 1292, 2649, 1218, 4024, 2306, 1723, 118...","[2324, 3735, 1182, 0, 1857, 620, 335, 2653, 37..."
2,"[3963, 745, 2424, 1734, 1808, 366, 3662, 1187,...","[3088, 148, 3443, 2766, 1211, 3141, 2786, 3530..."
3,"[1238, 1211, 1521, 2659, 4022, 3446, 2766, 131...","[3726, 2468, 3039, 1015, 64, 3765, 3726, 1229,..."
4,"[4016, 759, 1187, 4024, 1808, 366, 3455, 3292,...","[2362, 693, 1187, 2860, 1335, 790, 2199, 1332,..."
...,...,...
1699,"[2424, 1305, 2027, 1023, 3145, 3964, 3991, 115...","[2424, 1305, 3548, 983, 3765, 2356, 1218, 3743..."
1700,"[3446, 4022, 2322, 1211, 1814, 701, 2892, 2299...","[2351, 1163, 1182, 0, 3548, 983, 1218, 3726, 3..."
1701,"[3088, 148, 3431, 1814, 701]","[2324, 474, 1182, 0, 3296, 507, 3765, 4024, 18..."
1702,"[1218, 3765, 1652, 2860, 1229, 1188, 2860, 184...","[2892, 2934, 1332, 3538, 1314, 3757, 2325, 144..."


In [15]:
#Save data to csv and txt files
qa_index.to_csv("index.csv", index=False)
with open('vocab.txt', 'w') as f:
    for item in vocab_dict:
        f.write("%s\n" % item)
